In [25]:
!pip install mlflow
!pip install gensim==4.1.2

In [26]:
import csv
import mlflow
import functools
import gzip
import numpy as np
import pandas as pd
import torch
import tempfile
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from gensim import corpora

import torch.nn.functional as F
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from gensim.parsing import preprocessing
from torch.utils.data import Dataset, DataLoader, IterableDataset
from tqdm import tqdm, trange
from sklearn.metrics import average_precision_score
from sklearn.metrics import balanced_accuracy_score

In [27]:
data = pd.read_csv("C:/Users/Sofia Oliverio/EntregablesGrupo10/data_melli_challenge/train.csv") 

data = data[data['language']=='spanish']
data.shape

(10000000, 4)

In [28]:
class MeLiChallengeDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform
    
    def __len__(self):
        return self.dataset.shape[0]

    def __getitem__(self, item):
        if torch.is_tensor(item):
            item = item.tolist()  # Deal with list of items instead of tensor
        
        item = {
                    "data": self.dataset.iloc[item]["title"],
                    "target": self.dataset.iloc[item]["category"]                    
                }

        if self.transform:
            item = self.transform(item)
        
        return item

In [29]:
class TextPreprocess:
    def __init__(self, 
                 dataset, 
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        self.dictionary = corpora.Dictionary(
            dataset["title"].map(self._preprocess_string).tolist())
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()

        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })

        self.idx_to_target = sorted(dataset["category"].unique())
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target)}
        
    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)

    def encode_target(self, target): 
       return self.target_to_idx[target]

    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))

    def __call__(self, item):
        if isinstance(item["data"], str):
            data = self.encode_data(item["data"])
        else:
            data = [self.encode_data(d) for d in item["data"]]
      
        if isinstance(item["target"], str):
            target = self.encode_target(item["target"])
        else:
            target = [self.encode_target(t) for t in item["target"]]
    
        return {
            "data": data,
            "target": target
        }

In [30]:
preprocess = TextPreprocess(data)

In [31]:
train_indices, test_indices = train_test_split(data.index, test_size=0.2, random_state=42)
train_dataset = MeLiChallengeDataset(data.loc[train_indices].reset_index(drop=True), transform=preprocess)
test_dataset = MeLiChallengeDataset(data.loc[test_indices].reset_index(drop=True), transform=preprocess)

In [32]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
        return {
            "data": torch.LongTensor(data),
            "target": torch.FloatTensor(target)
        }

In [33]:
pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True,
                          collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False,
                         collate_fn=pad_sequences, drop_last=False)


In [34]:
class MELI_Categories_Classifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt", encoding="utf8") as fh:
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, 1)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

In [ ]:
mlflow.set_experiment("a_naive_experiment")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 50,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    model = MELI_Categories_Classifier("C:/Users/Sofia Oliverio/EntregablesGrupo10/data_melli_challenge/glove.6B.50d.txt", preprocess.dictionary, 50, True)
    loss = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(3):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            output = model(batch["data"])
            loss_value = loss(output, batch["target"].view(-1, 1))
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            running_loss.append(
                loss(output, batch["target"].view(-1, 1)).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric('test_bacc', balanced_accuracy_score(targets, np.round(predictions)), epoch)
    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            output = model(batch["data"])
            targets.extend(batch["target"].numpy())
            predictions.extend(output.squeeze().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

  1%|          | 367/62500 [00:19<57:32, 18.00it/s]


  1%|          | 762/62500 [00:39<48:06, 21.39it/s]


  2%|▏         | 1144/62500 [00:59<53:50, 18.99it/s]


  2%|▏         | 1489/62500 [01:17<45:10, 22.51it/s]


  3%|▎         | 1880/62500 [01:36<54:12, 18.64it/s]


...

 50%|████▉     | 31015/62500 [26:22<30:55, 16.97it/s]


 50%|█████     | 31301/62500 [26:43<31:29, 16.51it/s]


 51%|█████     | 31619/62500 [27:00<28:32, 18.03it/s]


 51%|█████     | 31937/62500 [27:18<26:57, 18.90it/s]


 51%|█████     | 31976/62500 [27:20<29:31, 17.23it/s]